#### מגישות:
ירדן סמואל 207386442

רננה פריד 209129394

githubקישור ל:
https://github.com/renanafried/Matala1

In [48]:
import requests
from bs4 import BeautifulSoup

def get_cars(keywords):
    car_list = []
    base_url = "https://www.ad.co.il/car?keyword=" + keywords
    page_number = 1

    while page_number < 9:
        url = f"{base_url}&page={page_number}"
        response = requests.get(url)
        if response.status_code != 200:
            break  # Exit if the page doesn't load correctly
        
        try:
            results_page = BeautifulSoup(response.content, 'html.parser')
            cars = results_page.find_all('div', {'class': 'card-body p-md-3'})
            if not cars:
                break  # Exit if no more cars are found
            
            for car in cars:
                car_link = "https://www.ad.co.il" + car.find('a').get('href')
                car_name = car.find('a').get_text().strip()
                if keywords in car_name:
                    
                    car_list.append(car_link)
            
            page_number += 1  # Move to the next page
        except Exception as e:
            print(f"Error: {e}")
            break
    
    return car_list

In [49]:
get_cars("מאזדה")

['https://www.ad.co.il/ad/16186152',
 'https://www.ad.co.il/ad/16185806',
 'https://www.ad.co.il/ad/16186411',
 'https://www.ad.co.il/ad/16168929',
 'https://www.ad.co.il/ad/16167062',
 'https://www.ad.co.il/ad/16166014',
 'https://www.ad.co.il/ad/16162970',
 'https://www.ad.co.il/ad/16137890',
 'https://www.ad.co.il/ad/16128563',
 'https://www.ad.co.il/ad/15952860',
 'https://www.ad.co.il/ad/15910677',
 'https://www.ad.co.il/ad/16116498',
 'https://www.ad.co.il/ad/16102966',
 'https://www.ad.co.il/ad/16068875',
 'https://www.ad.co.il/ad/16086340',
 'https://www.ad.co.il/ad/16067160',
 'https://www.ad.co.il/ad/16065478',
 'https://www.ad.co.il/ad/16050602',
 'https://www.ad.co.il/ad/16045395',
 'https://www.ad.co.il/ad/15905182',
 'https://www.ad.co.il/ad/16041416',
 'https://www.ad.co.il/ad/16035021',
 'https://www.ad.co.il/ad/16022117',
 'https://www.ad.co.il/ad/16018006',
 'https://www.ad.co.il/ad/16084476',
 'https://www.ad.co.il/ad/15899707',
 'https://www.ad.co.il/ad/16158490',
 

In [50]:
def get_car_info(car_link):
    from datetime import datetime, date
    car_list = list()
    car_info={}
    response = requests.get(car_link)
    if not response.status_code == 200:
        print ("not 200")
        return None
    try:
        result_page = BeautifulSoup(response.content,'html.parser')
        table=result_page.find('table',{'class':'table table-sm mb-4'}).find('tbody')
        if table:
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    key = cols[0].text.strip()
                    value = cols[1].text.strip()
                    car_info[key] = value
                    if 'שנה' in car_info:
                        car_info['Year'] = car_info.pop('שנה')
                        car_info['Year'] = int(car_info['Year'])
                    if 'יד' in car_info:
                        car_info['Hand'] = car_info.pop('יד')
                        car_info['Hand'] = int(car_info['Hand'])
                    if 'ת. הילוכים' in car_info:
                        car_info['Gear'] = car_info.pop('ת. הילוכים') 
                    if 'נפח' in car_info:
                        car_info['capacity_Engine'] = car_info.pop('נפח').replace(',', '')
                        car_info['capacity_Engine'] = int(car_info['capacity_Engine'])
                    if 'סוג מנוע' in car_info:
                        car_info['Engine_type'] = car_info.pop('סוג מנוע')
                    if 'בעלות קודמת' in car_info:
                        car_info['Prev_ownership'] = car_info.pop('בעלות קודמת')
                    if 'בעלות נוכחית' in car_info:
                        car_info['Curr_ownership'] = car_info.pop('בעלות נוכחית')
                    if 'אזור' in car_info:
                        car_info['Area'] = car_info.pop('אזור')
                    if 'עיר' in car_info:
                        car_info['City'] = car_info.pop('עיר')
                    if 'צבע' in car_info:
                        car_info['Color'] = car_info.pop('צבע')
                    if 'ק"מ' in car_info:
                        car_info['Km'] = car_info.pop('ק"מ').replace(',', '')
                        car_info['Km'] = int(car_info['Km']) 
                    if 'טסט עד' in car_info:
                        test_date_str = car_info.pop('טסט עד')
                        test_date = datetime.strptime(test_date_str, '%m/%Y')
                        today = datetime.now()
                        days_until_test = (test_date - today).days
                        car_info['Test'] = days_until_test

        car_info["manufactor"] = "מאזדה"
    
        model_and_price= result_page.find_all('h2',{'class':'card-title'})
        car_info['model']=model_and_price[0].get_text()
        car_info['Price']=float(model_and_price[1].get_text().strip('₪').replace(',', ''))
        
        picture= result_page.find_all('div',{'class':'justify-content-center px-1'})
        car_info['Pic_num']=int(len(picture))       
        
        Repub_date_and_Cre_date= result_page.find_all('div',{'class':'px-3'})
        cre_date=Repub_date_and_Cre_date[0].get_text().split()
        Repub_date=Repub_date_and_Cre_date[1].get_text().split()
        car_info['Cre_date']= cre_date[2].replace('/', '-')
        car_info['Repub_date']=Repub_date[3].replace('/', '-')
        car_info['Cre_date'] = datetime.strptime(car_info['Cre_date'], '%d-%m-%Y')
        car_info['Repub_date'] = datetime.strptime(car_info['Repub_date'], '%d-%m-%Y')

        car_info['description']=result_page.find('p',{'class':'text-word-break'}).get_text()
            
        return car_info
    except:
        return car_info
    

In [51]:
get_car_info('https://www.ad.co.il/ad/16168929')

{'Year': 2012,
 'Hand': 3,
 'Gear': 'אוטומטית',
 'capacity_Engine': 2000,
 'Engine_type': 'בנזין',
 'Km': 209,
 'Test': 57,
 'Color': 'שחור',
 'Prev_ownership': 'פרטית',
 'Curr_ownership': 'פרטית',
 'Area': 'קריות',
 'City': 'קרית אתא',
 'manufactor': 'מאזדה',
 'model': 'מאזדה 3',
 'Price': 29900.0,
 'Pic_num': 5,
 'Cre_date': datetime.datetime(2024, 4, 6, 0, 0),
 'Repub_date': datetime.datetime(2024, 4, 7, 0, 0),
 'description': 'הרכב במצב חדש שמור ומטופל לפי הספר '}

In [52]:
def get_all_cars(keywords):
    results = list()
    all_cars = get_cars(keywords)
    for car in all_cars:
        results.append(get_car_info(car))
    return(results)

In [53]:
mazda_cars = get_all_cars("מאזדה")
mazda_cars

[{'Year': 2008,
  'Hand': 3,
  'Gear': 'אוטומטית',
  'capacity_Engine': 2000,
  'Engine_type': 'בנזין',
  'Km': 195000,
  'Test': 118,
  'Color': 'שחור',
  'Prev_ownership': 'פרטית',
  'Curr_ownership': 'פרטית',
  'Area': 'בית שמש והסביבה',
  'City': 'בית שמש',
  'manufactor': 'מאזדה',
  'model': 'מאזדה 6',
  'Price': 16000.0,
  'Pic_num': 4,
  'Cre_date': datetime.datetime(2024, 5, 16, 0, 0),
  'Repub_date': datetime.datetime(2024, 6, 4, 0, 0),
  'description': 'מאזדה 6 שנת 2008 במצב מצוין מכאנית\nטסט עד סוף חודש 9 \n195,000 אלף קילומטר גמיש במחיר'},
 {'Year': 2015,
  'Hand': 4,
  'Gear': 'אוטומטית',
  'capacity_Engine': 1500,
  'Engine_type': 'בנזין',
  'Km': 152000,
  'Color': 'שחור',
  'Prev_ownership': 'פרטית',
  'Curr_ownership': 'פרטית',
  'Area': 'עכו - נהריה',
  'City': 'נהריה',
  'manufactor': 'מאזדה',
  'model': 'מאזדה 3',
  'Price': 43000.0,
  'Pic_num': 8,
  'Cre_date': datetime.datetime(2024, 5, 13, 0, 0),
  'Repub_date': datetime.datetime(2024, 5, 21, 0, 0),
  'descripti

#### יצירת דאטה פריים של הנתונים והשמה של משתנים קטגוריאליים

In [54]:
import pandas as pd
data= pd.DataFrame(mazda_cars)
data['Gear'] = pd.Categorical(data['Gear'])
data['Engine_type'] = pd.Categorical(data['Engine_type'])
data['Prev_ownership'] = pd.Categorical(data['Prev_ownership'])
data['Curr_ownership'] = pd.Categorical(data['Curr_ownership'])
data.head()

,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Test,Color,Prev_ownership,Curr_ownership,Area,City,manufactor,model,Price,Pic_num,Cre_date,Repub_date,description
0,2008,3,אוטומטית,2000,בנזין,195000.0,118.0,שחור,פרטית,פרטית,בית שמש והסביבה,בית שמש,מאזדה,מאזדה 6,16000.0,4,2024-05-16,2024-06-04,מאזדה 6 שנת 2008 במצב מצוין מכאנית\nטסט עד סוף...
1,2015,4,אוטומטית,1500,בנזין,152000.0,NaN,שחור,פרטית,פרטית,עכו - נהריה,נהריה,מאזדה,מאזדה 3,43000.0,8,2024-05-13,2024-05-21,"מאזדה 3 קומפורט 2015.\nיד 04 ברישיון\n152,000 ..."
2,2022,2,אוטומטית,2000,בנזין,30000.0,300.0,שחור,פרטית,פרטית,נתניה והסביבה,כפר יונה,מאזדה,מאזדה 3,133000.0,1,2024-05-19,2024-05-28,מאזדה 3 בלאק שנת 2022 יד 01 נסע 30 אלף ק״מ מטו...
3,2012,3,אוטומטית,2000,בנזין,209.0,57.0,שחור,פרטית,פרטית,קריות,קרית אתא,מאזדה,מאזדה 3,29900.0,5,2024-04-06,2024-04-07,הרכב במצב חדש שמור ומטופל לפי הספר
4,2007,4,אוטומטית,1600,בנזין,188000.0,57.0,בז' מטאלי,פרטית,פרטית,גדרה יבנה והסביבה,יבנה,מאזדה,מאזדה 3,8700.0,8,2024-04-04,2024-04-04,אחלה אוטו נוסע מצוין ידיים פרטיות צמיגים במצב ...


#### סינון השנים הרלוונטיות 2010-2024

In [55]:
data = data.loc[(data['Year'] >= 2010) & (data['Year'] <= 2024)]
data

,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Test,Color,Prev_ownership,Curr_ownership,Area,City,manufactor,model,Price,Pic_num,Cre_date,Repub_date,description
1,2015,4,אוטומטית,1500,בנזין,152000.0,NaN,שחור,פרטית,פרטית,עכו - נהריה,נהריה,מאזדה,מאזדה 3,43000.0,8,2024-05-13,2024-05-21,"מאזדה 3 קומפורט 2015.\nיד 04 ברישיון\n152,000 ..."
2,2022,2,אוטומטית,2000,בנזין,30000.0,300.0,שחור,פרטית,פרטית,נתניה והסביבה,כפר יונה,מאזדה,מאזדה 3,133000.0,1,2024-05-19,2024-05-28,מאזדה 3 בלאק שנת 2022 יד 01 נסע 30 אלף ק״מ מטו...
3,2012,3,אוטומטית,2000,בנזין,209.0,57.0,שחור,פרטית,פרטית,קריות,קרית אתא,מאזדה,מאזדה 3,29900.0,5,2024-04-06,2024-04-07,הרכב במצב חדש שמור ומטופל לפי הספר
5,2018,3,אוטומטית,2000,בנזין,52000.0,269.0,לבן פנינה,אחר,פרטית,אשדוד - אשקלון,אשקלון,מאזדה,מאזדה 3,74500.0,7,2024-04-03,2024-04-03,מאזדה 3 האצ׳בק ספורט 2018 למכירה\nנקנתה לפני ש...
6,2014,5,אוטומטית,2000,בנזין,194000.0,NaN,אפור,השכרה,פרטית,ירושלים והסביבה,ירושלים,מאזדה,מאזדה 6,35000.0,6,2024-04-01,2024-04-01,"מאזדה 6 2014 - יפה, חדש כמו ביום שיצא מהסולון...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2018,2,אוטומטית,2000,בנזין,104000.0,NaN,לבן,פרטית,פרטית,פרדס חנה - כרכור,חריש,מאזדה,מאזדה 3,85000.0,3,2023-05-23,2023-05-23,"הדגם המובחר והמוצלח עם גג נפתח, ג'אנטים 18, מו..."
375,2014,3,טיפטרוניק,1500,בנזין,144500.0,NaN,כחול כהה מטאלי,ליסינג,פרטית,רעננה - כפר סבא,כפר סבא,מאזדה,מאזדה 3,45000.0,7,2023-05-14,2023-05-14,מזדה 3 2014 מצב מצוין\n טסט עד חודש 8 \n144500...
376,2014,3,ידנית,1600,בנזין,170000.0,NaN,אפור עכבר,פרטית,פרטית,בית שמש והסביבה,בית שמש,מאזדה,מאזדה 3,50000.0,12,2023-05-02,2023-05-02,רכב מצב שמור מאוד ללא תאונות רכב חלק מהכל חסכו...
382,2019,2,אוטומטית,2000,בנזין,69000.0,149.0,כסוף,השכרה,פרטית,אשדוד - אשקלון,אשקלון,מאזדה,מאזדה MX-3,112000.0,6,2023-03-22,2023-03-22,למכירה מאזדה cx5 רמת גימור comfort עלייה לכביש...


#### בדיקת המשתנים לטיפוס הרצוי

In [56]:
print(data.dtypes)

Year                        int64
Hand                        int64
Gear                     category
capacity_Engine             int64
Engine_type              category
Km                        float64
Test                      float64
Color                      object
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
manufactor                 object
model                      object
Price                     float64
Pic_num                     int64
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
description                object
dtype: object


#### יצירת CSV

In [57]:
data.to_csv('C:\\Users\\רננה\\Desktop\\כריית וניתוח נתונים\\matala1.csv', index=False, encoding='utf-8-sig')